In [1]:
from visualization_export import VisualizationExport

### Data regeneration

The data should already exist, but this should be set to true if it needs to be regenerated.

In [2]:
regenerate_data = False
if regenerate_data:
    VisualizationExport.export_test_and_train()

Preparing data...
Building graph...
Running prediction...
Restoring model from models/crowd_net.ckpt-3021...
INFO:tensorflow:Summary name True person count is illegal; using True_person_count instead.
INFO:tensorflow:Summary name Predicted person count is illegal; using Predicted_person_count instead.
INFO:tensorflow:Summary name Person miscount is illegal; using Person_miscount instead.
INFO:tensorflow:Summary name Relative person miscount is illegal; using Relative_person_miscount instead.
1 images processed.
2 images processed.
3 images processed.
4 images processed.
5 images processed.
6 images processed.
7 images processed.
8 images processed.
9 images processed.
10 images processed.
11 images processed.
12 images processed.
13 images processed.
14 images processed.
15 images processed.
16 images processed.
17 images processed.
18 images processed.
19 images processed.
20 images processed.
21 images processed.
22 images processed.
23 images processed.
24 images processed.
25 image

In [3]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [1]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.palettes import Dark2
import numpy as np
import os

palette = Dark2[3]

hover = HoverTool(
        tooltips="""
        <div>
            <div>
                <img
                    src="@images" height="120" alt="@images" width="176"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
                <img
                    src="@predicted_images" height="120" alt="@predicted_images" width="176"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
                <img
                    src="@true_images" height="120" alt="@true_images" width="176"
                    style="float: left; margin: 0px 15px 15px 0px;"
                    border="2"
                ></img>
            </div>
            <div>
                <span style="font-size: 15px; color: #966;">[$index]</span>
            </div>
            <div>
                <span style="font-size: 15px;">Location</span>
                <span style="font-size: 10px; color: #696;">($x, $y)</span>
            </div>
        </div>
        """
    )

train_predicted_person_count = np.load('visualization/train/predicted_person_count.npy')
train_true_person_count = np.load('visualization/train/true_person_count.npy')
train_image_directory = 'visualization/train/original_images'
train_image_paths = ['{}/{}'.format(train_image_directory, file) for file in os.listdir(train_image_directory) if file.endswith('.jpeg')]
train_predicted_directory = 'visualization/train/predicted_label_images'
train_predicted_paths = ['{}/{}'.format(train_predicted_directory, file) for file in os.listdir(train_predicted_directory) if file.endswith('.jpeg')]
train_true_directory = 'visualization/train/true_label_images'
train_true_paths = ['{}/{}'.format(train_true_directory, file) for file in os.listdir(train_true_directory) if file.endswith('.jpeg')]

train_source = ColumnDataSource(
        data=dict(
            x=train_true_person_count,
            y=train_predicted_person_count,
            images=train_image_paths,
            predicted_images=train_predicted_paths,
            true_images=train_true_paths
        )
    )

test_predicted_person_count = np.load('visualization/test/predicted_person_count.npy')
test_true_person_count = np.load('visualization/test/true_person_count.npy')
test_image_directory = 'visualization/test/original_images'
test_image_paths = ['{}/{}'.format(test_image_directory, file) for file in os.listdir(test_image_directory) if file.endswith('.jpeg')]
test_predicted_directory = 'visualization/test/predicted_label_images'
test_predicted_paths = ['{}/{}'.format(test_predicted_directory, file) for file in os.listdir(test_predicted_directory) if file.endswith('.jpeg')]
test_true_directory = 'visualization/test/true_label_images'
test_true_paths = ['{}/{}'.format(test_true_directory, file) for file in os.listdir(test_true_directory) if file.endswith('.jpeg')]

test_source = ColumnDataSource(
        data=dict(
            x=test_true_person_count,
            y=test_predicted_person_count,
            images=test_image_paths,
            predicted_images=test_predicted_paths,
            true_images=test_true_paths
        )
    )

min_value = np.min((train_predicted_person_count, train_true_person_count, test_predicted_person_count, test_true_person_count))
max_value = np.max((train_predicted_person_count, train_true_person_count, test_predicted_person_count, test_true_person_count))


plot = figure(plot_width=800, plot_height=800, tools="pan,wheel_zoom,box_zoom,reset,save",
              title="Person Count", x_axis_label='True', y_axis_label='Predicted')
plot.add_tools(hover)
plot.line([min_value, max_value], [min_value, max_value], line_width=2, line_color=palette[2])
plot.circle('x', 'y', size=2, source=train_source, fill_color=palette[0], line_color=palette[0], fill_alpha=0.3, line_alpha=0.3)
plot.square('x', 'y', size=2, source=test_source, fill_color=palette[1], line_color=palette[1], fill_alpha=0.3, line_alpha=0.3)
show(plot)